# Sensitivity Kernel Computation for Seismic Tomography

This notebook demonstrates how to compute ray-theoretical sensitivity kernels using seisray.
These kernels are fundamental for seismic tomography inversions.

## Background
Sensitivity kernels quantify how much each grid cell in a model affects the travel time along a ray path.
For ray theory, the kernel represents the ray path density in each cell.

## Learning Objectives
- Understand sensitivity kernel concepts
- Compute kernels for different source-receiver geometries
- Visualize kernel patterns
- Prepare kernels for tomographic inversion

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from obspy.taup import TauPyModel
from seisray import SensitivityKernel

print("Loaded seisray for sensitivity kernel computation")
print("Using ObsPy for ray path calculations")

In [ ]:
# Setup tomography domain and grid
domain_bounds = (-500, 500, 0, 1000)  # x_min, x_max, y_min, y_max (km)
grid_size = (50, 50)  # nx, ny cells

# Initialize sensitivity kernel calculator
kernel_calc = SensitivityKernel(
    domain_bounds=domain_bounds,
    grid_size=grid_size,
    regularization=1e-4
)

print(f"Tomography domain: {domain_bounds} km")
print(f"Grid resolution: {grid_size[0]} x {grid_size[1]} cells")
print(f"Cell size: {kernel_calc.dx:.1f} x {kernel_calc.dy:.1f} km")

In [ ]:
# Define source and receiver arrays for a simple tomography setup
# This simulates a local earthquake tomography geometry

# Sources (earthquakes) - scattered throughout the domain
np.random.seed(42)  # For reproducibility
n_sources = 20
source_x = np.random.uniform(-400, 400, n_sources)
source_y = np.random.uniform(100, 800, n_sources)

# Receivers (stations) - surface array
n_receivers = 15
receiver_x = np.linspace(-450, 450, n_receivers)
receiver_y = np.full(n_receivers, 50)  # Near surface

print(f"Setup: {n_sources} sources, {n_receivers} receivers")
print(f"Total ray paths: {n_sources * n_receivers}")

# Visualize the geometry
plt.figure(figsize=(10, 8))
plt.scatter(source_x, source_y, c='red', s=50, marker='*', label='Sources (earthquakes)', alpha=0.7)
plt.scatter(receiver_x, receiver_y, c='blue', s=50, marker='^', label='Receivers (stations)', alpha=0.7)
plt.xlabel('X (km)')
plt.ylabel('Depth (km)')
plt.title('Local Earthquake Tomography Geometry')
plt.legend()
plt.grid(True, alpha=0.3)
plt.gca().invert_yaxis()  # Depth increases downward
plt.xlim(domain_bounds[0], domain_bounds[1])
plt.ylim(domain_bounds[3], domain_bounds[2])
plt.show()

In [ ]:
# Compute sensitivity kernels for all source-receiver pairs
print("Computing sensitivity kernels...")

# Initialize arrays to store results
all_kernels = []
ray_info = []

for i, (sx, sy) in enumerate(zip(source_x, source_y)):
    for j, (rx, ry) in enumerate(zip(receiver_x, receiver_y)):
        # Compute kernel for this source-receiver pair
        kernel = kernel_calc.compute_ray_kernel(
            source_pos=(sx, sy),
            receiver_pos=(rx, ry),
            ray_type='straight'  # Simple straight-ray approximation
        )

        all_kernels.append(kernel)
        ray_info.append({
            'source': (sx, sy),
            'receiver': (rx, ry),
            'distance': np.sqrt((sx - rx)**2 + (sy - ry)**2)
        })

    if (i + 1) % 5 == 0:
        print(f"  Processed {i + 1}/{n_sources} sources")

print(f"✅ Computed {len(all_kernels)} sensitivity kernels")

In [ ]:
# Visualize individual kernels
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.flatten()

# Show 6 example kernels
for i in range(6):
    idx = i * len(all_kernels) // 6  # Sample evenly
    kernel = all_kernels[idx]
    info = ray_info[idx]

    ax = axes[i]
    im = ax.imshow(kernel, extent=domain_bounds, origin='upper',
                   cmap='hot', aspect='auto')

    # Mark source and receiver
    ax.scatter(*info['source'], c='blue', s=100, marker='*', edgecolors='white', linewidth=1)
    ax.scatter(*info['receiver'], c='green', s=100, marker='^', edgecolors='white', linewidth=1)

    ax.set_title(f"Kernel {idx+1}\nDistance: {info['distance']:.1f} km")
    ax.set_xlabel('X (km)')
    ax.set_ylabel('Depth (km)')

    plt.colorbar(im, ax=ax, shrink=0.8, label='Sensitivity')

plt.tight_layout()
plt.suptitle('Individual Ray-Theoretical Sensitivity Kernels', y=1.02, fontsize=14)
plt.show()

In [ ]:
# Compute and visualize the cumulative sensitivity (ray coverage)
total_sensitivity = np.sum(all_kernels, axis=0)

plt.figure(figsize=(12, 8))
im = plt.imshow(total_sensitivity, extent=domain_bounds, origin='upper',
                cmap='viridis', aspect='auto')

# Overlay source and receiver positions
plt.scatter(source_x, source_y, c='red', s=30, marker='*',
           alpha=0.8, edgecolors='white', linewidth=0.5, label='Sources')
plt.scatter(receiver_x, receiver_y, c='yellow', s=30, marker='^',
           alpha=0.8, edgecolors='black', linewidth=0.5, label='Receivers')

plt.colorbar(im, label='Total Ray Coverage')
plt.xlabel('X (km)')
plt.ylabel('Depth (km)')
plt.title('Cumulative Sensitivity (Ray Coverage) for Tomography')
plt.legend()
plt.grid(True, alpha=0.3, color='white')
plt.show()

print(f"Ray coverage statistics:")
print(f"  Maximum sensitivity: {np.max(total_sensitivity):.2f}")
print(f"  Mean sensitivity: {np.mean(total_sensitivity):.2f}")
print(f"  Cells with coverage: {np.sum(total_sensitivity > 0)} / {total_sensitivity.size}")
print(f"  Coverage percentage: {100 * np.sum(total_sensitivity > 0) / total_sensitivity.size:.1f}%")

In [ ]:
# Prepare kernel matrix for tomographic inversion
# Each row represents one ray path, each column represents one model cell

n_rays = len(all_kernels)
n_cells = grid_size[0] * grid_size[1]

# Flatten kernels to create the sensitivity matrix G
G = np.array([kernel.flatten() for kernel in all_kernels])

print(f"Sensitivity matrix G: {G.shape}")
print(f"  {G.shape[0]} ray paths (observations)")
print(f"  {G.shape[1]} model cells (parameters)")
print(f"  Sparsity: {100 * np.sum(G == 0) / G.size:.1f}% zero entries")

# Visualize matrix structure
plt.figure(figsize=(10, 6))
plt.imshow(G, aspect='auto', cmap='Blues', interpolation='nearest')
plt.xlabel('Model Cell Index')
plt.ylabel('Ray Path Index')
plt.title('Sensitivity Matrix Structure (G)')
plt.colorbar(label='Sensitivity')
plt.show()

# Matrix conditioning
condition_number = np.linalg.cond(G.T @ G)
print(f"\nMatrix conditioning:")
print(f"  Condition number of G^T G: {condition_number:.2e}")
if condition_number > 1e12:
    print("  ⚠️  Matrix is poorly conditioned - consider regularization")
else:
    print("  ✅ Matrix conditioning is acceptable")

## Summary

This notebook demonstrated:

1. **Sensitivity Kernel Computation**: Using seisray to compute ray-theoretical kernels for tomography
2. **Geometry Setup**: Typical local earthquake tomography with sources and receivers
3. **Individual Kernels**: Visualization of sensitivity patterns for different ray paths
4. **Ray Coverage**: Assessment of model resolution through cumulative sensitivity
5. **Inversion Preparation**: Creating the sensitivity matrix G for tomographic inversion

### Next Steps
- Use this sensitivity matrix in a tomographic inversion (e.g., with scipy or specialized codes)
- Apply regularization techniques for stable solutions
- Assess resolution through synthetic tests
- Extend to more complex ray theory (curved rays, multiple phases)

### Key Points
- Ray-theoretical kernels provide the foundation for tomographic imaging
- Geometry design critically affects resolution
- Matrix conditioning indicates inversion stability
- seisray focuses on kernel computation while leveraging ObsPy for ray calculations